In [ ]:
#todo: 
#author check for Jr, one letter key and auth

In [1]:
import codecs
import pandas as pd
import re
import collections

In [2]:
with codecs.open('isr_raw.txt','r','utf-8') as f:
    content = f.readlines()
f.close()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [3]:
articles=[]
article={}
for i in content:
    if i[0:2]=='%0':
        if len(article)>0:
            articles.append(article)
        article={}; article['AF']=''; article['DE']='';line=0
        article['PT']=i[3:]
    elif i[0:2]=='%A':
        article['AF']=(article['AF']+i[3:] if article['AF']=='' else article['AF']+'; '+i[3:])
    elif i[0:2]=='%T':article['TI']=i[3:]
    elif i[0:2]=='%D':article['PY']=i[3:]
    elif i[0:2]=='%J':article['SO']=i[3:]
    elif i[0:2]=='%D':article['PY']=i[3:]
    elif i[0:2]=='%P':article['PG']=i[3:]
    elif i[0:2]=='%V':article['VL']=i[3:]
    elif i[0:2]=='%N':article['IS']=i[3:]
    elif i[0:2]=='%U':article['UR']=i[3:]
    elif i[0:2]=='%X':article['AB']=i[3:]
    elif i[0:2]=='%K':article['DE']='; '.join(i[3:].split(','))

In [4]:
df=pd.DataFrame(articles)
df.PG.fillna('-',inplace=True)
df['BP']=df['PG'].apply(lambda x: x.split('-')[0])
df['EP']=df['PG'].apply(lambda x: x.split('-')[1])
df=df[~df.AB.isnull()]
df=df[df.AF!='']
df.shape

(795, 13)

In [8]:
DE=[]
for index, row in df.iterrows():
    keywords=row['DE'].split(';')
    keywords_=''
    for keyword in keywords:
        keyword=re.sub(r'\(.+?\)', '', keyword).strip()
        words=keyword.split()
        words_=''
        for idx, word in enumerate(words):
            if idx>0:
                if len(word)>1:
                    if (word[0].isupper() and word[1].islower()):
                        word=word.lower()
            words_=words_+' '+word
        if keywords_=='':
            keywords_=words_        
        else:
            keywords_=keywords_+'; '+words_   
    DE.append(keywords_)
len(DE)

795

In [7]:
a=[]
for row in DE:
    a=a+(row.split(';'))
a=[i.strip() for i in a]
counter=collections.Counter(a)
counter.most_common(10)

[(u'electronic commerce', 25),
 (u'social media', 19),
 (u'information technology', 18),
 (u'economics of IS', 16),
 (u'decision support systems', 14),
 (u'business value of IT', 13),
 (u'trust', 12),
 (u'user-generated content', 12),
 (u'information systems', 11),
 (u'innovation', 11)]

In [12]:
df['DE']=DE
df.to_csv('isr_1990_2016_dec.csv',columns=['AF','TI','DE','AB','SO','VL','PY','IS','BP','EP'],encoding='utf-8')